# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [46]:
!git clone https://github.com/JulianEgererHub/JulsLerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!pip install -e /content/JulsLerobot


fatal: destination path 'JulsLerobot' already exists and is not an empty directory.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Obtaining file:///content/JulsLerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.4.1-0.editable-py3-none-any.whl size=15654 sha256=3f122e5d3fc67f2627046341d3af87b479b4fa7a530c7c6f5eaa6e68ea4364d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-wmf8rmpz/wheels/85/49/68/7e51aa0807c8e869e24ec7fea7f1464581ad60b4b86ab63

In [48]:
!pip install -e /content/JulsLerobot



Obtaining file:///content/JulsLerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.4.1-0.editable-py3-none-any.whl size=15654 sha256=e70643a4f0f53c8e734ef53b14faba966f4d75dbf3ad8a462dcc8a866dde061a
  Stored in directory: /tmp/pip-ephem-wheel-cache-wzdhidmy/wheels/85/49/68/7e51aa0807c8e869e24ec7fea7f1464581ad60b4b86ab63e93
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.4.1
    Uninstalling lerobot-0.4.1:
      Successfully uninstalled lerobot-0.4.1


## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ju-egerer (ju-egerer-technical-university-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Start training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [17]:
!nvidia-smi



Fri Nov 21 23:36:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   46C    P8             17W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [38]:
from huggingface_hub import login
login()



In [39]:
from huggingface_hub import HfApi

api = HfApi()
print(api.whoami())  # should print your username


{'type': 'user', 'id': '691a2184e2baf7728cad0f2b', 'name': 'JulianEgerer', 'fullname': 'Julian Egerer', 'email': 'ju.egerer@gmail.com', 'emailVerified': True, 'canPay': False, 'billingMode': 'postpaid', 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/9253f0a56feccf92f0070d5acdd9d2fe.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'colab', 'role': 'write', 'createdAt': '2025-11-22T00:17:11.017Z'}}}


In [43]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

ds = LeRobotDataset.from_hub(
    repo_id="JulianEgerer/my_first_dataset",
    repo_type="dataset"  # <- crucial
)



ModuleNotFoundError: No module named 'lerobot'

In [41]:
!python JulsLerobot/src/lerobot/scripts/lerobot_train.py  --dataset.repo_id=JulianEgerer/my_first_dataset   --policy.type=act   --output_dir=outputs/train/policy2  --job_name=act_so101_test   --policy.device=cuda   --wandb.enable=true   --policy.repo_id=JulianEgerer/my_first_policy

None
INFO 2025-11-22 00:17:46 ot_train.py:163 {'batch_size': 8,
 'checkpoint_path': None,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'affine': {'kwargs': {'degrees': [-5.0,
                                                                            5.0],
                                                                'translate': [0.05,
                                                                              0.05]},
                                                     'type': 'RandomAffine',
                                                     'weight': 1.0},
                                          'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                      

In [ ]:
!python JulsLerobot/src/lerobot/scripts/lerobot_train.py \
  --dataset.repo_id=JulianEgerer/my_first_dataset_corrected \
  --policy.type=act \
  --output_dir=outputs/train/teeeest\
  --job_name=test \
  --policy.device=cuda \
  --policy.repo_id=JulianEgerer/my_first_policy \
  --policy.push_to_hub=true \
  --wandb.enable=true

None
INFO 2025-11-21 23:56:29 ot_train.py:163 {'batch_size': 8,
 'checkpoint_path': None,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'affine': {'kwargs': {'degrees': [-5.0,
                                                                            5.0],
                                                                'translate': [0.05,
                                                                              0.05]},
                                                     'type': 'RandomAffine',
                                                     'weight': 1.0},
                                          'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                      

In [ ]:
!huggingface-cli upload ${HF_USER}/il_sim_test0 \
  /content/lerobot/outputs/train/il_sim_test0/checkpoints/last/pretrained_model